In [ ]:
# ==========================================================
# Hybrid CNN-ViT Architecture for Knee Osteoarthritis
# Stage 1: EfficientNet (Local Features)
# Stage 2: Vision Transformer (Global Context)
# Stage 3: Attention Fusion Module
# Stage 4: Hierarchical Classification (Binary→Ternary→5-Class)
# ==========================================================

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import AdamW
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (confusion_matrix, classification_report, 
                             roc_auc_score, roc_curve, auc, 
                             precision_recall_fscore_support)
from sklearn.preprocessing import label_binarize
from scipy import signal
from scipy.ndimage import gaussian_filter
import warnings
warnings.filterwarnings('ignore')

# ---------------------------
# Configuration
# ---------------------------
CONFIG = {
    'WORK_DIR': './',
    'EPOCHS': 100,
    'BATCH_SIZE': 8,
    'IMG_SIZE': 224,
    'NUM_CLASSES': 5,
    'LEARNING_RATE': 1e-4,
    'WEIGHT_DECAY': 1e-5,
    'PATCH_SIZE': 16,
    'NUM_PATCHES': (224 // 16) ** 2,  # 196 patches
    'PROJECTION_DIM': 512,
    'NUM_HEADS': 12,
    'TRANSFORMER_LAYERS': 4,
    'MLP_HEAD_UNITS': [2048, 1024],
}

PATHS = {
    'train': '/kaggle/input/koa-dataset/dataset/train',
    'val': '/kaggle/input/koa-dataset/dataset/val',
    'test': '/kaggle/input/koa-dataset/dataset/test'
}

CLASS_NAMES = ['KL-0', 'KL-1', 'KL-2', 'KL-3', 'KL-4']

# ---------------------------
# 1. ADVANCED PREPROCESSING
# ---------------------------

class KalmanFilter1D:
    """1D Kalman filter for noise reduction"""
    def __init__(self, process_variance=1e-5, measurement_variance=1e-1):
        self.process_variance = process_variance
        self.measurement_variance = measurement_variance
        self.posteri_estimate = 0.0
        self.posteri_error_estimate = 1.0
    
    def update(self, measurement):
        # Prediction
        priori_estimate = self.posteri_estimate
        priori_error_estimate = self.posteri_error_estimate + self.process_variance
        
        # Update
        blending_factor = priori_error_estimate / (priori_error_estimate + self.measurement_variance)
        self.posteri_estimate = priori_estimate + blending_factor * (measurement - priori_estimate)
        self.posteri_error_estimate = (1 - blending_factor) * priori_error_estimate
        
        return self.posteri_estimate

def apply_kalman_filter(image):
    """Apply Kalman filter to each row/column for noise reduction"""
    filtered = np.zeros_like(image, dtype=np.float32)
    
    # Filter rows
    for i in range(image.shape[0]):
        kf = KalmanFilter1D()
        for j in range(image.shape[1]):
            filtered[i, j] = kf.update(image[i, j])
    
    # Filter columns
    for j in range(image.shape[1]):
        kf = KalmanFilter1D()
        for i in range(image.shape[0]):
            filtered[i, j] = kf.update(filtered[i, j])
    
    return filtered

def laplacian_sharpen(image, kernel_size=3, strength=1.5):
    """Laplacian sharpening for edge enhancement"""
    # Ensure image is in correct format
    img_float = image.astype(np.float32)
    
    laplacian_kernel = np.array([[-1, -1, -1],
                                  [-1,  9, -1],
                                  [-1, -1, -1]], dtype=np.float32)
    
    sharpened = cv2.filter2D(img_float, -1, laplacian_kernel)
    result = img_float + (sharpened - img_float) * (strength - 1.0)
    
    return np.clip(result, 0, 255).astype(np.uint8)

def high_pass_filter(image, kernel_size=15):
    """High-pass filter for texture emphasis"""
    # Ensure image is float32 for processing
    img_float = image.astype(np.float32)
    
    # Apply Gaussian blur (low-pass)
    low_pass = cv2.GaussianBlur(img_float, (kernel_size, kernel_size), 0)
    
    # Subtract from original to get high-pass
    high_pass = img_float - low_pass
    
    # Add back to original for enhancement
    enhanced = img_float + (high_pass * 0.5)
    
    return np.clip(enhanced, 0, 255).astype(np.uint8)

def detect_knee_roi(image):
    """
    Detect knee joint ROI using edge detection and morphology
    (Simplified version - in production, use YOLOv8)
    """
    # Convert to grayscale if needed
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # Apply CLAHE for better contrast
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    
    # Edge detection
    edges = cv2.Canny(enhanced, 50, 150)
    
    # Morphological operations to find joint region
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    dilated = cv2.dilate(closed, kernel, iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Get largest contour (assumed to be knee joint)
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        # Add padding
        padding = 20
        x = max(0, x - padding)
        y = max(0, y - padding)
        w = min(image.shape[1] - x, w + 2 * padding)
        h = min(image.shape[0] - y, h + 2 * padding)
        
        return (x, y, w, h)
    
    # If no contour found, return center crop
    h, w = image.shape[:2]
    crop_size = min(h, w) // 2
    x = (w - crop_size) // 2
    y = (h - crop_size) // 2
    return (x, y, crop_size, crop_size)

def advanced_preprocess(img_path, target_size=224):
    """
    Complete preprocessing pipeline:
    1. Load and initial CLAHE
    2. Laplacian sharpening
    3. High-pass filtering
    4. Kalman filter noise reduction
    5. ROI extraction
    6. Resize and normalize
    """
    # Load image
    img = cv2.imread(img_path)
    if img is None:
        return None
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Step 1: Initial CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    
    # Step 2: Laplacian sharpening
    sharpened = laplacian_sharpen(enhanced, strength=1.3)
    
    # Step 3: High-pass filtering
    high_pass = high_pass_filter(sharpened, kernel_size=15)
    
    # Step 4: Kalman filter (applied on normalized image)
    normalized = high_pass.astype(np.float32) / 255.0
    filtered = apply_kalman_filter(normalized)
    filtered = (filtered * 255).astype(np.uint8)
    
    # Step 5: ROI extraction
    x, y, w, h = detect_knee_roi(filtered)
    roi = filtered[y:y+h, x:x+w]
    
    # Step 6: Resize to target size
    resized = cv2.resize(roi, (target_size, target_size), interpolation=cv2.INTER_LANCZOS4)
    
    # Convert to RGB and normalize to [-1, 1]
    rgb = cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)
    final = (rgb.astype(np.float32) / 127.5) - 1.0
    
    return final, (x, y, w, h)

# ---------------------------
# 2. DATA LOADING
# ---------------------------

def build_dataset(data_dir, target_size=224):
    """Build dataset with advanced preprocessing"""
    filepaths, labels, rois = [], [], []
    
    for klass in sorted(os.listdir(data_dir)):
        klass_path = os.path.join(data_dir, klass)
        if not os.path.isdir(klass_path):
            continue
        
        klass_idx = int(klass)
        
        for fname in os.listdir(klass_path):
            img_path = os.path.join(klass_path, fname)
            
            # Preprocess
            result = advanced_preprocess(img_path, target_size)
            if result is not None:
                preprocessed_img, roi_coords = result
                filepaths.append(preprocessed_img)
                labels.append(klass_idx)
                rois.append(roi_coords)
    
    return np.array(filepaths), np.array(labels), rois

# ---------------------------
# 3. VISION TRANSFORMER COMPONENTS
# ---------------------------

class PatchExtractor(layers.Layer):
    """Extract patches from images"""
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size
    
    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

class PatchEncoder(layers.Layer):
    """Encode patches with position embeddings"""
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )
    
    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

class TransformerBlock(layers.Layer):
    """Transformer encoder block"""
    def __init__(self, projection_dim, num_heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.attn = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=dropout
        )
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.mlp = keras.Sequential([
            layers.Dense(mlp_dim, activation=tf.nn.gelu),
            layers.Dropout(dropout),
            layers.Dense(projection_dim),
            layers.Dropout(dropout),
        ])
    
    def call(self, encoded_patches, training):
        # Multi-head attention
        x1 = self.norm1(encoded_patches)
        attention_output = self.attn(x1, x1, training=training)
        x2 = layers.Add()([attention_output, encoded_patches])
        
        # MLP
        x3 = self.norm2(x2)
        x3 = self.mlp(x3, training=training)
        encoded_patches = layers.Add()([x3, x2])
        
        return encoded_patches

# ---------------------------
# 4. CNN FEATURE EXTRACTOR
# ---------------------------

def build_cnn_backbone(input_shape=(224, 224, 3)):
    """EfficientNetB0 for local feature extraction"""
    inputs = layers.Input(shape=input_shape)
    
    # EfficientNetB0 backbone
    backbone = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_tensor=inputs
    )
    
    # Get features from multiple scales
    # We'll use the output before global pooling
    features = backbone.output  # Shape: (None, 7, 7, 1792)
    
    return Model(inputs, features, name='CNN_Backbone')

# ---------------------------
# 5. ATTENTION FUSION MODULE
# ---------------------------

class AttentionFusion(layers.Layer):
    """Fuse CNN and ViT features with learned attention"""
    def __init__(self, fusion_dim=1024):
        super().__init__()
        self.fusion_dim = fusion_dim
        
        # Projection layers
        self.cnn_proj = layers.Dense(fusion_dim, activation='relu')
        self.vit_proj = layers.Dense(fusion_dim, activation='relu')
        
        # Attention weights
        self.cnn_attention = layers.Dense(1, activation='sigmoid')
        self.vit_attention = layers.Dense(1, activation='sigmoid')
        
        # Final fusion
        self.fusion_layer = layers.Dense(fusion_dim, activation='relu')
        self.dropout = layers.Dropout(0.3)
    
    def call(self, cnn_features, vit_features, training=False):
        # Project to same dimension
        cnn_proj = self.cnn_proj(cnn_features)
        vit_proj = self.vit_proj(vit_features)
        
        # Compute attention weights
        cnn_weight = self.cnn_attention(cnn_proj)
        vit_weight = self.vit_attention(vit_proj)
        
        # Normalize weights
        total_weight = cnn_weight + vit_weight
        cnn_weight = cnn_weight / (total_weight + 1e-8)
        vit_weight = vit_weight / (total_weight + 1e-8)
        
        # Weighted fusion
        fused = cnn_weight * cnn_proj + vit_weight * vit_proj
        fused = self.fusion_layer(fused)
        fused = self.dropout(fused, training=training)
        
        return fused, cnn_weight, vit_weight

# ---------------------------
# 6. HIERARCHICAL CLASSIFIER
# ---------------------------

class HierarchicalClassifier(layers.Layer):
    """
    Hierarchical classification:
    Level 1: Binary (Healthy vs OA)
    Level 2: Ternary (Mild vs Moderate vs Severe)
    Level 3: Fine-grained (5-class KL grade)
    """
    def __init__(self):
        super().__init__()
        
        # Level 1: Binary classifier (0 vs 1-4)
        self.binary_head = keras.Sequential([
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(2, activation='softmax', name='binary_output')
        ])
        
        # Level 2: Ternary classifier (0-1 vs 2-3 vs 4)
        self.ternary_head = keras.Sequential([
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(3, activation='softmax', name='ternary_output')
        ])
        
        # Level 3: Fine-grained classifier (0-4)
        self.fine_head = keras.Sequential([
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(5, activation='softmax', name='fine_output')
        ])
    
    def call(self, features, training=False):
        binary_pred = self.binary_head(features, training=training)
        ternary_pred = self.ternary_head(features, training=training)
        fine_pred = self.fine_head(features, training=training)
        
        return binary_pred, ternary_pred, fine_pred

# ---------------------------
# 7. COMPLETE HYBRID MODEL
# ---------------------------

def build_hybrid_model(config=CONFIG):
    """
    Build complete hybrid CNN-ViT model
    """
    img_size = config['IMG_SIZE']
    patch_size = config['PATCH_SIZE']
    num_patches = config['NUM_PATCHES']
    projection_dim = config['PROJECTION_DIM']
    num_heads = config['NUM_HEADS']
    transformer_layers = config['TRANSFORMER_LAYERS']
    
    inputs = layers.Input(shape=(img_size, img_size, 3))
    
    # ========== Stage 1: CNN Feature Extraction ==========
    cnn_backbone = build_cnn_backbone((img_size, img_size, 3))
    cnn_features = cnn_backbone(inputs)
    
    # Global average pooling for CNN features
    cnn_gap = layers.GlobalAveragePooling2D()(cnn_features)
    
    # ========== Stage 2: Vision Transformer ==========
    # Extract patches
    patches = PatchExtractor(patch_size)(inputs)
    
    # Encode patches
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    
    # Transformer blocks
    for _ in range(transformer_layers):
        encoded_patches = TransformerBlock(
            projection_dim=projection_dim,
            num_heads=num_heads,
            mlp_dim=projection_dim * 4,
            dropout=0.1
        )(encoded_patches, training=True)
    
    # Global representation from ViT
    vit_representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    vit_gap = layers.GlobalAveragePooling1D()(vit_representation)
    
    # ========== Stage 3: Attention Fusion ==========
    fusion_module = AttentionFusion(fusion_dim=1024)
    fused_features, cnn_weight, vit_weight = fusion_module(cnn_gap, vit_gap)
    
    # ========== Stage 4: Hierarchical Classification ==========
    hierarchical_classifier = HierarchicalClassifier()
    binary_output, ternary_output, fine_output = hierarchical_classifier(fused_features)
    
    # Build model
    model = Model(
        inputs=inputs,
        outputs={
            'binary': binary_output,
            'ternary': ternary_output,
            'fine': fine_output
        },
        name='Hybrid_CNN_ViT_KOA'
    )
    
    return model

# ---------------------------
# 8. CUSTOM LOSS FUNCTIONS
# ---------------------------

def hierarchical_loss(y_true_fine, y_pred_binary, y_pred_ternary, y_pred_fine, 
                      alpha=0.3, beta=0.3, gamma=0.4):
    """
    Combined hierarchical loss
    """
    # Convert fine labels to hierarchical labels
    # Binary: 0 -> class 0, 1-4 -> class 1
    y_true_binary = tf.cast(y_true_fine > 0, tf.int32)
    y_true_binary = tf.one_hot(y_true_binary, 2)
    
    # Ternary: 0-1 -> class 0, 2-3 -> class 1, 4 -> class 2
    y_true_ternary = tf.where(y_true_fine <= 1, 0,
                              tf.where(y_true_fine <= 3, 1, 2))
    y_true_ternary = tf.one_hot(y_true_ternary, 3)
    
    # Fine: 0-4 -> classes 0-4
    y_true_fine_onehot = tf.one_hot(y_true_fine, 5)
    
    # Compute losses
    loss_binary = keras.losses.categorical_crossentropy(y_true_binary, y_pred_binary)
    loss_ternary = keras.losses.categorical_crossentropy(y_true_ternary, y_pred_ternary)
    loss_fine = keras.losses.categorical_crossentropy(y_true_fine_onehot, y_pred_fine)
    
    # Combined loss
    total_loss = alpha * loss_binary + beta * loss_ternary + gamma * loss_fine
    
    return total_loss

# ---------------------------
# 9. TRAINING
# ---------------------------

def train_model(model, train_data, val_data, config=CONFIG):
    """Train the hybrid model"""
    
    X_train, y_train = train_data
    X_val, y_val = val_data
    
    # Custom training loop for hierarchical loss
    optimizer = AdamW(learning_rate=config['LEARNING_RATE'], 
                      weight_decay=config['WEIGHT_DECAY'])
    
    train_loss_tracker = keras.metrics.Mean(name='train_loss')
    val_loss_tracker = keras.metrics.Mean(name='val_loss')
    train_acc_tracker = keras.metrics.CategoricalAccuracy(name='train_acc')
    val_acc_tracker = keras.metrics.CategoricalAccuracy(name='val_acc')
    
    @tf.function
    def train_step(x, y):
        with tf.GradientTape() as tape:
            predictions = model(x, training=True)
            loss = hierarchical_loss(
                y, 
                predictions['binary'],
                predictions['ternary'],
                predictions['fine']
            )
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        train_loss_tracker.update_state(loss)
        train_acc_tracker.update_state(tf.one_hot(y, 5), predictions['fine'])
        
        return loss
    
    @tf.function
    def val_step(x, y):
        predictions = model(x, training=False)
        loss = hierarchical_loss(
            y,
            predictions['binary'],
            predictions['ternary'],
            predictions['fine']
        )
        
        val_loss_tracker.update_state(loss)
        val_acc_tracker.update_state(tf.one_hot(y, 5), predictions['fine'])
        
        return loss
    
    # Training loop
    best_val_loss = float('inf')
    patience = 15
    patience_counter = 0
    
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(config['EPOCHS']):
        print(f"\nEpoch {epoch + 1}/{config['EPOCHS']}")
        
        # Reset metrics (use reset_state, not reset_states)
        train_loss_tracker.reset_state()
        val_loss_tracker.reset_state()
        train_acc_tracker.reset_state()
        val_acc_tracker.reset_state()
        
        # Training
        num_batches = len(X_train) // config['BATCH_SIZE']
        for batch in range(num_batches):
            start_idx = batch * config['BATCH_SIZE']
            end_idx = start_idx + config['BATCH_SIZE']
            
            x_batch = X_train[start_idx:end_idx]
            y_batch = y_train[start_idx:end_idx]
            
            train_step(x_batch, y_batch)
            
            if batch % 10 == 0:
                print(f"Batch {batch}/{num_batches} - "
                      f"Loss: {train_loss_tracker.result():.4f} - "
                      f"Acc: {train_acc_tracker.result():.4f}", end='\r')
        
        # Validation
        num_val_batches = len(X_val) // config['BATCH_SIZE']
        for batch in range(num_val_batches):
            start_idx = batch * config['BATCH_SIZE']
            end_idx = start_idx + config['BATCH_SIZE']
            
            x_batch = X_val[start_idx:end_idx]
            y_batch = y_val[start_idx:end_idx]
            
            val_step(x_batch, y_batch)
        
        # Log results
        train_loss = train_loss_tracker.result()
        val_loss = val_loss_tracker.result()
        train_acc = train_acc_tracker.result()
        val_acc = val_acc_tracker.result()
        
        history['train_loss'].append(float(train_loss))
        history['val_loss'].append(float(val_loss))
        history['train_acc'].append(float(train_acc))
        history['val_acc'].append(float(val_acc))
        
        print(f"\nTrain Loss: {train_loss:.4f} - Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            model.save_weights('best_hybrid_model.weights.h5')
            print("✓ Model saved!")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered after {epoch + 1} epochs")
                break
    
    return model, history

# ---------------------------
# 10. COMPREHENSIVE EVALUATION
# ---------------------------

def evaluate_comprehensive(model, X_test, y_test):
    """
    Comprehensive evaluation with multiple metrics
    """
    print("\n" + "="*70)
    print("COMPREHENSIVE MODEL EVALUATION")
    print("="*70)
    
    # Get predictions
    predictions = model.predict(X_test, batch_size=16, verbose=1)
    
    # Extract fine-grained predictions
    y_pred_probs = predictions['fine']
    y_pred = np.argmax(y_pred_probs, axis=1)
    
    # Binary predictions
    y_pred_binary = np.argmax(predictions['binary'], axis=1)
    y_true_binary = (y_test > 0).astype(int)
    
    # Ternary predictions
    y_pred_ternary = np.argmax(predictions['ternary'], axis=1)
    y_true_ternary = np.where(y_test <= 1, 0, np.where(y_test <= 3, 1, 2))
    
    # ========== Binary Classification Metrics ==========
    print("\n📊 BINARY CLASSIFICATION (Healthy vs OA):")
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true_binary, y_pred_binary, average='binary'
    )
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    
    # ========== Ternary Classification Metrics ==========
    print("\n📊 TERNARY CLASSIFICATION (Mild vs Moderate vs Severe):")
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true_ternary, y_pred_ternary, average='weighted'
    )
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")
    
    # ========== Fine-grained Metrics ==========
    print("\n📊 FINE-GRAINED CLASSIFICATION (5-class KL Grade):")
    print("\nPer-Class Metrics:")
    precision, recall, f1, support = precision_recall_fscore_support(
        y_test, y_pred, average=None
    )
    
    for i, class_name in enumerate(CLASS_NAMES):
        print(f"\n  {class_name}:")
        print(f"    Precision: {precision[i]:.4f}")
        print(f"    Recall:    {recall[i]:.4f}")
        print(f"    F1-Score:  {f1[i]:.4f}")
        print(f"    Support:   {support[i]}")
    
    # Overall metrics
    precision_avg, recall_avg, f1_avg, _ = precision_recall_fscore_support(
        y_test, y_pred, average='weighted'
    )
    print(f"\n  Overall (Weighted):")
    print(f"    Precision: {precision_avg:.4f}")
    print(f"    Recall:    {recall_avg:.4f}")
    print(f"    F1-Score:  {f1_avg:.4f}")
    
    # ========== Confusion Matrix ==========
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
    plt.title('Confusion Matrix - Fine-grained Classification')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig('confusion_matrix_hybrid.png', dpi=150)
    plt.show()
    
    # ========== ROC-AUC Curves ==========
    print("\n📈 Computing ROC-AUC Curves...")
    
    # Binarize labels for multi-class ROC
    y_test_bin = label_binarize(y_test, classes=range(5))
    
    # Compute ROC curve and AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for i in range(5):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Plot ROC curves
    plt.figure(figsize=(12, 8))
    colors = ['blue', 'green', 'orange', 'red', 'purple']
    
    for i, color in enumerate(colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                label=f'{CLASS_NAMES[i]} (AUC = {roc_auc[i]:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves - Multi-class Classification', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig('roc_curves_hybrid.png', dpi=150)
    plt.show()
    
    print("\n✅ Evaluation Complete!")
    
    return {
        'y_pred': y_pred,
        'y_pred_probs': y_pred_probs,
        'y_pred_binary': y_pred_binary,
        'y_pred_ternary': y_pred_ternary,
        'roc_auc': roc_auc
    }

# ---------------------------
# 11. ATTENTION VISUALIZATION
# ---------------------------

def visualize_attention_maps(model, X_samples, y_samples, num_samples=5):
    """
    Visualize attention heatmaps for interpretability
    """
    print("\n" + "="*70)
    print("ATTENTION VISUALIZATION")
    print("="*70)
    
    # Create a model that outputs intermediate attention weights
    # We'll visualize both CNN and ViT attention
    
    fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))
    
    for idx in range(min(num_samples, len(X_samples))):
        img = X_samples[idx:idx+1]
        true_label = y_samples[idx]
        
        # Get predictions
        predictions = model.predict(img, verbose=0)
        pred_label = np.argmax(predictions['fine'])
        
        # Original image
        original = ((img[0] + 1.0) * 127.5).astype(np.uint8)
        
        # For visualization, we'll create a simple gradient-based attention map
        with tf.GradientTape() as tape:
            tape.watch(img)
            predictions = model(img, training=False)
            pred_score = predictions['fine'][0, pred_label]
        
        # Get gradients
        grads = tape.gradient(pred_score, img)
        
        # Create attention map
        grads_abs = tf.abs(grads)
        attention_map = tf.reduce_mean(grads_abs, axis=-1)[0].numpy()
        
        # Normalize
        attention_map = (attention_map - attention_map.min()) / (attention_map.max() - attention_map.min() + 1e-8)
        
        # Apply colormap
        attention_colored = cv2.applyColorMap(
            (attention_map * 255).astype(np.uint8),
            cv2.COLORMAP_JET
        )
        
        # Overlay
        overlay = cv2.addWeighted(original, 0.6, attention_colored, 0.4, 0)
        
        # Plot
        axes[idx, 0].imshow(original)
        axes[idx, 0].set_title(f'Original\nTrue: {CLASS_NAMES[true_label]}')
        axes[idx, 0].axis('off')
        
        axes[idx, 1].imshow(attention_map, cmap='jet')
        axes[idx, 1].set_title('Attention Map')
        axes[idx, 1].axis('off')
        
        axes[idx, 2].imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
        axes[idx, 2].set_title(f'Overlay\nPred: {CLASS_NAMES[pred_label]}')
        axes[idx, 2].axis('off')
        
        # Show prediction probabilities
        probs = predictions['fine'][0].numpy()
        axes[idx, 3].barh(CLASS_NAMES, probs, color='steelblue')
        axes[idx, 3].set_xlim([0, 1])
        axes[idx, 3].set_title('Prediction Probabilities')
        axes[idx, 3].set_xlabel('Probability')
    
    plt.tight_layout()
    plt.savefig('attention_visualization.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Attention maps generated!")

# ---------------------------
# 12. MAIN EXECUTION
# ---------------------------

def main():
    """
    Complete pipeline execution
    """
    print("\n" + "="*70)
    print("HYBRID CNN-ViT FOR KNEE OSTEOARTHRITIS CLASSIFICATION")
    print("="*70)
    
    # ========== STEP 1: Load and Preprocess Data ==========
    print("\n[STEP 1/5] Loading and preprocessing data...")
    print("⚠️  This may take several minutes due to advanced preprocessing...")
    
    print("\nProcessing training data...")
    X_train, y_train, train_rois = build_dataset(PATHS['train'], CONFIG['IMG_SIZE'])
    
    print("Processing validation data...")
    X_val, y_val, val_rois = build_dataset(PATHS['val'], CONFIG['IMG_SIZE'])
    
    print("Processing test data...")
    X_test, y_test, test_rois = build_dataset(PATHS['test'], CONFIG['IMG_SIZE'])
    
    print(f"\n✓ Dataset loaded:")
    print(f"  Train: {len(X_train)} samples")
    print(f"  Val:   {len(X_val)} samples")
    print(f"  Test:  {len(X_test)} samples")
    
    # ========== STEP 2: Build Model ==========
    print("\n[STEP 2/5] Building hybrid CNN-ViT model...")
    
    model = build_hybrid_model(CONFIG)
    
    print(f"\n✓ Model built successfully!")
    print(f"  Total parameters: {model.count_params():,}")
    
    # Model summary
    model.summary()
    
    # ========== STEP 3: Train Model ==========
    print("\n[STEP 3/5] Training model...")
    
    trained_model, history = train_model(
        model,
        train_data=(X_train, y_train),
        val_data=(X_val, y_val),
        config=CONFIG
    )
    
    # Load best weights
    trained_model.load_weights('best_hybrid_model.h5')
    print("\n✓ Best model weights loaded!")
    
    # ========== STEP 4: Evaluate Model ==========
    print("\n[STEP 4/5] Evaluating model on test set...")
    
    eval_results = evaluate_comprehensive(trained_model, X_test, y_test)
    
    # ========== STEP 5: Visualize Attention ==========
    print("\n[STEP 5/5] Generating attention visualizations...")
    
    # Select diverse samples for visualization
    sample_indices = []
    for class_idx in range(5):
        class_indices = np.where(y_test == class_idx)[0]
        if len(class_indices) > 0:
            sample_indices.append(class_indices[0])
    
    if len(sample_indices) >= 5:
        X_samples = X_test[sample_indices[:5]]
        y_samples = y_test[sample_indices[:5]]
        
        visualize_attention_maps(trained_model, X_samples, y_samples, num_samples=5)
    
    # ========== Training History Visualization ==========
    print("\n📊 Plotting training history...")
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss
    axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Accuracy
    axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
    axes[1].plot(history['val_acc'], label='Validation Accuracy', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=150)
    plt.show()
    
    # ========== Summary ==========
    print("\n" + "="*70)
    print("✅ PIPELINE COMPLETED SUCCESSFULLY!")
    print("="*70)
    print(f"\n📁 Generated files:")
    print(f"  • best_hybrid_model.h5")
    print(f"  • confusion_matrix_hybrid.png")
    print(f"  • roc_curves_hybrid.png")
    print(f"  • attention_visualization.png")
    print(f"  • training_history.png")
    
    return trained_model, history, eval_results

# ---------------------------
# RUN PIPELINE
# ---------------------------

if __name__ == "__main__":
    model, history, results = main()


HYBRID CNN-ViT FOR KNEE OSTEOARTHRITIS CLASSIFICATION

[STEP 1/5] Loading and preprocessing data...
⚠️  This may take several minutes due to advanced preprocessing...

Processing training data...
Processing validation data...
Processing test data...

✓ Dataset loaded:
  Train: 5778 samples
  Val:   826 samples
  Test:  1656 samples

[STEP 2/5] Building hybrid CNN-ViT model...

✓ Model built successfully!
  Total parameters: 67,431,343


Model: "Hybrid_CNN_ViT_KOA"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_31      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_extractor_3   │ (None, None, 768) │          0 │ input_layer_31[0… │
│ (PatchExtractor)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_encoder_3     │ (None, 196, 512)  │    494,080 │ patch_extractor_… │
│ (PatchEncoder)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_… │ (None, 196, 512)  │ 14,703,616 │ patch_encoder_3[… │
│ (TransformerBlock)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_… │ (None, 196, 512)  │ 14,703,616 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_… │ (None, 196, 512)  │ 14,703,616 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_… │ (None, 196, 512)  │ 14,703,616 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ CNN_Backbone        │ (None, 7, 7,      │  4,049,571 │ input_layer_31[0… │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 196, 512)  │      1,024 │ transformer_bloc… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ CNN_Backbone[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_fusion_3  │ [(None, 1024),    │  2,888,706 │ global_average_p… │
│ (AttentionFusion)   │ (None, 1), (None, │            │ global_average_p… │
│                     │ 1)]               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hierarchical_class… │ [(None, 2),       │  1,183,498 │ attention_fusion… │
│ (HierarchicalClass… │ (None, 3), (None, │            │                   │
│                     │ 5)]               │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 67,431,343 (257.23 MB)

 Trainable params: 67,389,320 (257.07 MB)

 Non-trainable params: 42,023 (164.16 KB)


[STEP 3/5] Training model...

Epoch 1/100


E0000 00:00:1764322511.083023      38 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inHybrid_CNN_ViT_KOA_1/CNN_Backbone_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Batch 720/722 - Loss: 0.2560 - Acc: 0.9263
Train Loss: 0.2569 - Train Acc: 0.9256
Val Loss: 4.8101 - Val Acc: 0.0473
✓ Model saved!

Epoch 2/100
Batch 720/722 - Loss: 0.3257 - Acc: 0.8925
Train Loss: 0.3264 - Train Acc: 0.8913
Val Loss: 21001.1406 - Val Acc: 0.1286

Epoch 3/100
Batch 720/722 - Loss: 0.3386 - Acc: 0.8696
Train Loss: 0.3393 - Train Acc: 0.8686
Val Loss: 87564.0469 - Val Acc: 0.1286

Epoch 4/100
Batch 720/722 - Loss: 0.2982 - Acc: 0.8797
Train Loss: 0.2980 - Train Acc: 0.8798
Val Loss: 8917.6465 - Val Acc: 0.0303

Epoch 5/100
